In [27]:
import pandas as pd

sub = pd.read_csv('../ 잡케어 추천 알고리즘/job_care/sample_submission.csv')
train = pd.read_csv('../ 잡케어 추천 알고리즘/job_care/train.csv')
test = pd.read_csv('../ 잡케어 추천 알고리즘/job_care/test.csv')

# person_rn :  사용자번호
# contents_rn :  컨텐츠번호
# contents_open_dt :  컨텐츠 열람 일시

In [63]:
y = train['target']
train = train.drop(['id','target','person_rn','contents_rn','contents_open_dt'],axis=1)
test = test.drop(['id','person_rn','contents_rn','contents_open_dt'],axis=1)

In [64]:
train =pd.get_dummies(train)
test =pd.get_dummies(test)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501951 entries, 0 to 501950
Data columns (total 30 columns):
 #   Column                  Non-Null Count   Dtype
---  ------                  --------------   -----
 0   d_l_match_yn            501951 non-null  bool 
 1   d_m_match_yn            501951 non-null  bool 
 2   d_s_match_yn            501951 non-null  bool 
 3   h_l_match_yn            501951 non-null  bool 
 4   h_m_match_yn            501951 non-null  bool 
 5   h_s_match_yn            501951 non-null  bool 
 6   person_attribute_a      501951 non-null  int64
 7   person_attribute_a_1    501951 non-null  int64
 8   person_attribute_b      501951 non-null  int64
 9   person_prefer_c         501951 non-null  int64
 10  person_prefer_d_1       501951 non-null  int64
 11  person_prefer_d_2       501951 non-null  int64
 12  person_prefer_d_3       501951 non-null  int64
 13  person_prefer_e         501951 non-null  int64
 14  person_prefer_f         501951 non-null  int64
 15  

In [ ]:
bool_col=train.select_dtypes('bool').columns

train[bool_col] = train[bool_col].astype('int')
test[bool_col] = test[bool_col].astype('int')



In [81]:
train['d_sum'] = train.iloc[:,:3].sum(axis=1)

In [94]:
for content in train['d_sum'].items():  
    if content ==3 :
        train['d_match_all'] = 1
    else: 
        train['d_match_all'] = 0

In [ ]:
train

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
train_sc = sc.fit_transform(train)
test_sc = sc.transform(test)

## Optuna

In [5]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier

In [6]:
X_tr, X_val, y_tr, y_val = train_test_split(train_sc, y, stratify= y , test_size = 0.33, random_state=42)

lgbm = LGBMClassifier(random_state =42,objective = 'binary',metric = 'binary_logloss')

def objective(trial: Trial) -> float:
    params_lgb = {
        "random_state": 42,
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1, log=True), 
        'n_estimators': trial.suggest_int('n_estimators', 8, 1024, step=1, log=True), 
        "metric": "f1_score",
        'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 50, step=1, log=False), 
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
    }
    X_tr, X_val, y_tr, y_val = train_test_split(train , y, test_size=0.2,random_state=42)

    model = LGBMClassifier(**params_lgb)
    model.fit(X_tr,y_tr,
         eval_set=[(X_tr, y_tr), (X_val, y_val)],
        verbose=False,
    )
    pred = model.predict(X_val)
    score = f1_score(y_val,pred)
    
    return score



In [7]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="lgbm_parameter_opt",
    direction="maximize",
    sampler=sampler)

[I 2021-12-10 10:22:27,176] A new study created in memory with name: lgbm_parameter_opt


In [8]:
study.optimize(objective, n_trials=10)

[I 2021-12-10 10:23:07,998] Trial 0 finished with value: 0.6210562005736435 and parameters: {'learning_rate': 0.0013292918943162175, 'n_estimators': 804, 'class_weight': 'balanced', 'min_child_samples': 16, 'max_depth': 4, 'num_leaves': 16}. Best is trial 0 with value: 0.6210562005736435.
[I 2021-12-10 10:23:15,540] Trial 1 finished with value: 0.6340353196713572 and parameters: {'learning_rate': 0.0396760507705299, 'n_estimators': 144, 'class_weight': 'balanced', 'min_child_samples': 49, 'max_depth': 17, 'num_leaves': 56}. Best is trial 1 with value: 0.6340353196713572.
[I 2021-12-10 10:23:17,208] Trial 2 finished with value: 0.4121310949529512 and parameters: {'learning_rate': 0.0003511356313970409, 'n_estimators': 18, 'class_weight': None, 'min_child_samples': 27, 'max_depth': 6, 'num_leaves': 158}. Best is trial 1 with value: 0.6340353196713572.
[I 2021-12-10 10:23:19,057] Trial 3 finished with value: 0.5514221334126951 and parameters: {'learning_rate': 0.00026210878782654407, 'n_e

In [9]:
print(study.best_params,'\n')

{'learning_rate': 0.0396760507705299, 'n_estimators': 144, 'class_weight': 'balanced', 'min_child_samples': 49, 'max_depth': 17, 'num_leaves': 56} 



In [ ]:
study.

In [70]:
model = LGBMClassifier(**study.best_params,random_state=42)
model.fit(X_tr,y_tr)
fin_pred=model.predict(test_sc)



In [72]:
sub['target'] = fin_pred
sub.to_csv('sample2.csv',index=False)  

## voting 

- lgbm + rf + xgb
- voting model도 Optuna로 파라미터 찾고 최종 결합

In [10]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from xgboost import XGBClassifier

In [18]:
def objective_2(trial: Trial) -> float:
    params_rf = {
        "random_state": 42,
        'n_estimators': trial.suggest_int('n_estimators', 8, 1024, step=1, log=True), 
        "max_depth": trial.suggest_int("max_depth", 1, 20) }
    
    X_tr, X_val, y_tr, y_val = train_test_split(train , y, test_size=0.2,random_state=42)
    
    model = RandomForestClassifier(**params_rf)
    model.fit(X_tr, y_tr)
    pred = model.predict(X_val)
    score = f1_score(y_val,pred)
    return score

sampler = TPESampler(seed=42)
study_rf = optuna.create_study(
    study_name="rf_parameter_opt",
    direction="maximize",
    sampler=sampler)

study_rf.optimize(objective_2, n_trials=10)
print(study_rf.best_params,'\n')


model2 = RandomForestClassifier(**study.best_params,random_state=42)
model2.fit(X_tr,y_tr)
fin_pred=model2.predict(test_sc)

[I 2021-12-10 10:28:01,289] A new study created in memory with name: rf_parameter_opt
[I 2021-12-10 10:28:45,759] Trial 0 finished with value: 0.6312909551824747 and parameters: {'n_estimators': 47, 'max_depth': 20}. Best is trial 0 with value: 0.6312909551824747.
[I 2021-12-10 10:31:20,625] Trial 1 finished with value: 0.6223669965241506 and parameters: {'n_estimators': 274, 'max_depth': 12}. Best is trial 0 with value: 0.6312909551824747.
[I 2021-12-10 10:31:26,304] Trial 2 finished with value: 0.5857368116404862 and parameters: {'n_estimators': 16, 'max_depth': 4}. Best is trial 0 with value: 0.6312909551824747.
[I 2021-12-10 10:31:34,811] Trial 3 finished with value: 0.6113516020561974 and parameters: {'n_estimators': 10, 'max_depth': 18}. Best is trial 0 with value: 0.6312909551824747.
[I 2021-12-10 10:33:10,514] Trial 4 finished with value: 0.6282587215930683 and parameters: {'n_estimators': 144, 'max_depth': 15}. Best is trial 0 with value: 0.6312909551824747.
[I 2021-12-10 10:3

{'n_estimators': 47, 'max_depth': 20} 



TypeError: __init__() got an unexpected keyword argument 'learning_rate'

In [25]:
model = LGBMClassifier(**study.best_params,random_state=42)
model2 = RandomForestClassifier(n_estimators= 47, max_depth=20)

voting_model = VotingClassifier(estimators=[ ('lgbm', model),
                                            ('RF', model2) ], voting='soft')


    
voting_model.fit(X_tr, y_tr)
pred_voting = voting_model.predict(X_val)
print('보팅 분류기: {0: .4f}'.format(f1_score(y_val, pred_voting)))


voting_pred =voting_model.predict(test_sc)

sub['target'] = voting_pred
sub.to_csv('sample3.csv',index=False)  

보팅 분류기:  0.6358


## Submit History
- 1등 : 0.69
- #### sample2 
    - drop: 'id','target','person_rn','contents_rn','contents_open_dt'
    - MinMax, lgbm, optuna -> 0.63445 (38등)
    
- #### sample3
    - drop: 'id','target','person_rn','contents_rn','contents_open_dt'
    - Mimax, lgbm+rf_optuna (voting) -> 0.6323 // 더 낮아졌음 !